# NSMC-Sentimental-Analysis

본 과정은 Ainize Workspace에서 beomi/KcELECTRA-base 모델을 NSMC(Naver sentiment movie corpus) 데이터 셋으로 학습하는 과정입니다.

최종 목표는 학습된 모델을 통해 Sentimental Analysis를 하는 것입니다.

### Parameters 설정

학습에 사용될 parameters(학습/평가 데이터 경로, 에폭 등)를 설정합니다.

In [9]:
args = {
    'train_data_path': './nsmc/ratings_train.txt',
    'val_data_path': './nsmc/ratings_test.txt',
    'save_path': './model',
    'max_epochs': 1,
    'model_path': 'beomi/KcELECTRA-base',
    'batch_size': 32,
    'learning_rate': 5e-5,
    'warmup_ratio': 0.0,
    'max_seq_len': 128
}

### 데이터 처리

우선 데이터를 살펴보겠습니다. 

데이터에는 영화 리뷰와 리뷰에 대한 라벨이 포함되어 있습니다. 평점이 10, 9인 리뷰에 대해서는 긍정(1), 평점이 1, 2, 3, 4인 리뷰에 대해서는 부정(0)으로 라벨링되어 있습니다.

In [14]:
df = pd.read_csv(args["train_data_path"], sep='\t')
df

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1
...,...,...,...
149995,6222902,인간이 문제지.. 소는 뭔죄인가..,0
149996,8549745,평점이 너무 낮아서...,1
149997,9311800,이게 뭐요? 한국인은 거들먹거리고 필리핀 혼혈은 착하다?,0
149998,2376369,청춘 영화의 최고봉.방황과 우울했던 날들의 자화상,1


다운로드 받은 nsmc 데이터로부터 학습 데이터를 만들기 위해 Pytorch Dataset을 만들어 줍니다. 

데이터에 NaN 값과 중복 된 값이 포함되어 있어 이를 제거하는 작업과 입력으로 들어온 문자열을 max_length까지 자르는 작업을 진행하였습니다.

In [2]:
import pandas as pd
import torch

from torch.utils.data import Dataset

class NSMCDataset(Dataset):
    def __init__(self, csv_file, tokenizer, max_length):
        df = pd.read_csv(csv_file, sep='\t')
        # NaN 값 제거
        df = df.dropna(axis=0)
        # 중복 제거
        df.drop_duplicates(subset=['document'], inplace=True)
        self.input_ids = tokenizer.batch_encode_plus(
            df['document'].to_list(),
            padding='max_length',
            max_length=max_length,
            return_tensors='pt',
            return_token_type_ids=False,
            return_attention_mask=False,
            truncation=True,
        )['input_ids']
        self.labels = torch.LongTensor(df['label'])

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.labels[idx]

### Model 및 Tokenizer 불러오기

학습에 사용할 모델과 Tokenizer 파일을 Huggingface에서 불러옵니다.

모델을 불러오면 경고가 발생하는 데 이는 ElectraForSequenceClassification를 fine-tuning 하기 전에는 모델의 성능이 좋지 않을 것이라고 알려줍니다.

In [3]:
from transformers import ElectraForSequenceClassification, ElectraTokenizerFast

model = ElectraForSequenceClassification.from_pretrained(args['model_path'])
tokenizer = ElectraTokenizerFast.from_pretrained(args['model_path'])

Some weights of the model checkpoint at beomi/KcELECTRA-base were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense.weight', 'discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense.bias']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at beomi/KcELECTRA-base and are newly initialized: ['classifier.out_proj.bias', 'classifier

### 학습

모델을 학습합니다. 학습에 사용할 파라메터는 args에서 정의 하였습니다

In [4]:
from transformers import AdamW, get_linear_schedule_with_warmup
from tqdm.notebook import tqdm

def train(model, train_dataloader, args):
    model.train()
    model.to('cuda')
    global_total_step = len(train_dataloader) * args['max_epochs']
    global_step = 0
    optimizer = AdamW(model.parameters(), lr=args['learning_rate'], weight_decay=0.0)
    scheduler = get_linear_schedule_with_warmup(optimizer,
                                                num_warmup_steps=0,
                                                num_training_steps=global_total_step)
    with tqdm(total=global_total_step, unit='step') as t:
        total = 0
        total_loss = 0
        total_correct = 0
        for epoch in range(args['max_epochs']):
            for batch in train_dataloader:
                global_step += 1
                b_input_ids = batch[0].to('cuda', non_blocking=True)
                b_labels = batch[1].to('cuda', non_blocking=True)
                model.zero_grad(set_to_none=True)
                outputs = model(
                    input_ids=b_input_ids,
                    labels=b_labels
                )
                loss, logits = outputs.loss, outputs.logits

                loss.backward()
                optimizer.step()
                scheduler.step()

                preds = logits.detach().argmax(dim=-1).cpu().numpy()
                out_label_ids = b_labels.detach().cpu().numpy()
                total_correct += (preds == out_label_ids).sum()

                batch_loss = loss.item() * len(b_input_ids)

                total += len(b_input_ids)
                total_loss += batch_loss

                t.set_postfix(loss='{:.6f}'.format(batch_loss),
                              accuracy='{:.2f}'.format(total_correct / total * 100))
                t.update(1)
                del b_input_ids
                del outputs
                del loss

In [5]:
from torch.utils.data import DataLoader

train_data_set = NSMCDataset(args['train_data_path'], tokenizer, args['max_seq_len'])
train_data_loader = DataLoader(
    dataset=train_data_set,
    batch_size=args['batch_size'],
    pin_memory=True,
    shuffle=True,
    )

In [6]:
train(model, train_data_loader, args)

  0%|          | 0/4569 [00:00<?, ?step/s]

In [10]:
model.save_pretrained(args['save_path'])

### Smaple Data 넣어 보기

학습된 모델에 Smaple Data를 넣어 결과를 확인해보겠습니다.

In [31]:
# 평점 10
pos_text = '이방원을 다룬 드라마중 최고였다고 자부함. 진짜 이방원을 보여준 듯이 연기와 인물묘사나 주변상황이 재밌었고 스토리도 진부하지 않았음. 다시 이런드라마를 볼수 있을지~ 진짜 이런 드라마하나 또 나왔음 함.'
# 평점 0
neg_text = '핵노잼 후기보고 낙였네 방금보고왔는데 개실망 재미없어요'

In [32]:
pos_input_vector = tokenizer.encode(pos_text, return_tensors='pt').to('cuda')
pos_pred = model(input_ids=pos_input_vector, labels=None).logits.argmax(dim=-1).tolist()
print(f'{pos_text} : {pos_pred[0]}')

neg_input_vector = tokenizer.encode(neg_text, return_tensors='pt').to('cuda')
neg_pred = model(input_ids=neg_input_vector, labels=None).logits.argmax(dim=-1).tolist()
print(f'{neg_text} : {neg_pred[0]}')

이방원을 다룬 드라마중 최고였다고 자부함. 진짜 이방원을 보여준 듯이 연기와 인물묘사나 주변상황이 재밌었고 스토리도 진부하지 않았음. 다시 이런드라마를 볼수 있을지~ 진짜 이런 드라마하나 또 나왔음 함. : 1
핵노잼 후기보고 낙였네 방금보고왔는데 개실망 재미없어요 : 0


지금까지 beomi/KcELECTRA-base 모델을 NSMC(Naver sentiment movie corpus)데이터 셋으로 학습하는 과정을 진행해보았습니다.

beomi/kcbert-base, beomi/kcbert-large 등 다른 모델들도 NSMC로 학습하여 어떤 결과가 나오는 지 확인해보세요!